# Initialization


In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pandas
import seaborn as sns
from sklearn import preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from collections import Counter
import re
import string
import matplotlib.cm as cm
from matplotlib import rcParams
from prettytable import PrettyTable
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
import seaborn as sns
#load data done
import os

!pip install googletrans
!pip install langdetect

from googletrans import Translator
from langdetect import detect

#display data
column_names = ["review_description", "rating"]
train_data = pd.read_excel("/content/train.xlsx", names=column_names)
test_data = pd.read_csv("/content/test _no_label.csv",encoding="utf-8",dtype={'review_description': str})
print(len(train_data))
print(len(test_data))

train_data[train_data.isnull().any(axis=1)].head()
test_data[train_data.isnull().any(axis=1)].head()
print(np.sum(train_data.isnull().any(axis=1)))   #done
print(train_data.isnull().any(axis=0))

print(np.sum(test_data.isnull().any(axis=1)))   #done
print(test_data.isnull().any(axis=0))

from IPython.display import display, HTML
#display(train_data)
#display(test_data)

#print(train_data.head())
#print(test_data.head())
#print("***************************************************************************")
train_data["review_description"] = train_data["review_description"].astype(str)
test_data["review_description"] = test_data["review_description"].astype(str)

train_data["review_description"] = train_data["review_description"].fillna({"review_description":"مجهول"})
#######################################  NOTE  #############################################################
test_data["review_description"]  =test_data["review_description"].fillna({"review_description":"مجهول"})

print(train_data.head())
print(test_data.head())

for letter in '#.][!XR':
    train_data["review_description"] = train_data["review_description"].astype(str).str.replace(letter,'')
    test_data["review_description"] = test_data["review_description"].astype(str).str.replace(letter,'')

train_data.replace('', np.nan, inplace=True)

train_data["review_description"] = train_data["review_description"].fillna({"review_description":"مجهول"})
print("                 ***************** فارع        ****************")
print(train_data["review_description"].isnull().sum())
#train_data.dropna(inplace=True)
#print(len(train_data))


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.5 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15715 sha256=ea8a149e0fd92c21326812d6a9251d3cbf9eed85c8b27d66910aa563d281978e
  Stored in directory: /root/.cache/pip/wheels/b3/81/ea/8b030407f8ebfc2f857814e086bb22ca2d4fea1a7be63652ab
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling cha

# preprocessing

In [4]:
#preprocessing
#from translate import Translator
from googletrans import Translator
import nltk
from googletrans import Translator
#from translate import Translator
#done but with limimted trials
def translate_EtoA(english_text):
    # Create an instance of the Translator
    translator = Translator(service_urls=['translate.google.com'],to_lang="ar")
    # Translate English text to Arabic
    translation = translator.translate(english_text)
    # Access the translated text
    arabic_text = translation
    return arabic_text

#!pip install emojis
#!pip install emoji
from nltk.corpus import stopwords
import re
import csv
!pip install emojis
!pip install emoji
import emojis
import emoji
nltk.download('stopwords')

arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

def remove_stop_words(text):
    stop_words = set(stopwords.words('arabic')) - {'نعم','لا', 'ليس', 'ليست', 'مش', 'ما','غير','أقبل','ليس','ليسا','ليست','ليستا','ليسوا','لست','لستم','لستما','لستن','لسن','لسنا','واو'}
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    return " ".join([word for word in tokens if word not in stop_words])

def build_emoji_dictionary(csv_file):
    emoji_dict = {}
    with open(csv_file, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        for row in reader:
            emoji = row[0]
            text = row[1]
            emoji_dict[emoji] = text
    return emoji_dict

def replace_emojis(emoji_dict, sentence):
    emojis = emoji.emoji_list(sentence)
    for emo in emojis:
        if emo['emoji'] in emoji_dict:
            # Replace the emoji with the corresponding text surrounded by asterisks
            sentence = sentence.replace(emo['emoji'], '*' + emoji_dict[emo['emoji']] + '* ')
    return sentence

def remove_emojis(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def decode_emojis(text):
    # Replace emojis with Arabic words
    decoded_text = emojis.decode(text)
    return decoded_text

def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

emoji_dict = build_emoji_dictionary("/content/emojis.csv") #Assuming the CSV file is in the same directory
# in Arabic syllabels are represented by small signs above or below
# each character, we will remove them
# and see if they affect our prediction model
!pip install pyarabic
import pyarabic.araby as araby

def remove_diactrics(text):
    return araby.strip_diacritics(text)

def processPost(text,emoji_dict):
    #Replace @username with empty string
    text = re.sub(r"\s+", " ", str(text).strip())

    text = re.sub('@[^\s]+', ' ', text)
    #Convert www.* or https?://* to " "
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text)
    #remove nums
    text = re.sub(r'\d+', '', text)
    #Replace #word with word
    text = re.sub(r'#([^\s]+)', r'\1', text)
    # remove punctuations
    text= remove_punctuations(text)
    text = normalize_arabic(text)
    #handle emojes
    text = replace_emojis(emoji_dict, text)

    text= remove_punctuations(text)
    # Translate English to Arabic
    #text = translate_EtoA(text)  done but with limimted trials

    # remove repeated letters
    text = remove_repeating_char(text)
    text = remove_stop_words(text)
    #print(text)
    text = remove_diactrics(text)
    return text

train_data["review_description"] = train_data["review_description"].fillna({"review_description":"مجهول"})
print("              *****************    فارع        ****************")
print(train_data["review_description"].isnull().sum())
#print(train_data.shape[0])
train_data["review_description"] = train_data["review_description"].apply(lambda x: processPost(x,emoji_dict))
test_data["review_description"] = test_data["review_description"].apply(lambda x: processPost(x,emoji_dict))


train_data.replace('', np.nan, inplace=True)
train_data["review_description"] = train_data["review_description"].fillna({"review_description":"مجهول"})
print("                 ***************** فارع        ****************")
print(train_data["review_description"].isnull().sum())
#train_data.dropna(inplace=True)
#print(len(train_data))

test_data.replace('', np.nan, inplace=True)
test_data["review_description"] = test_data["review_description"].fillna({"review_description":"مجهول"})
print("                 ***************** فارع        ****************")
print(test_data["review_description"].isnull().sum())
#test_data.dropna(inplace=True)
#print(len(test_data))
#6 nana cases to be asked 3 number cases  ,1 case :'نناا'  , cases 'ععععع'

tokenizer = RegexpTokenizer(r'\w+')
train_data["review_description"] = train_data["review_description"].apply(str)
train_data["review_description"] = train_data["review_description"].apply(tokenizer.tokenize)


#stemming   to try
from nltk.stem.isri import ISRIStemmer
!pip install nltk
stemmer = ISRIStemmer()

def stem(text):
    stemmed = []
    for word in text:
        stemmed.append(stemmer.stem(word))
    return stemmed

from nltk.stem import SnowballStemmer
def engstem(text):
    e_stemmer = SnowballStemmer("english")
    stemmed = []
    for word in text:
        stemmed.append(e_stemmer.stem(word))
    return stemmed


#train_data["review_description"] = train_data["review_description"].apply(stem)
train_data["review_description"] = train_data["review_description"].apply(engstem)

# Join the stemmed tokens back into a string
train_data["review_description"] = train_data["review_description"].apply(lambda x: ' '.join(x))

train_data["review_description"]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


              *****************    فارع        ****************
0
                 ***************** فارع        ****************
0
                 ***************** فارع        ****************
0


0        شركه زباله سواقين بتبرشم مفيش حتي رقم لشكاوي ا...
1        خدمه الدفع طريق الكي نت توقفت عندي اصبح فقط ال...
2        تطبيق غبي جاري حذفه عاملين اكواد خصم نستخدمها ...
3        فعلا تطبيق متاز امكانيه يتيح لمستخدم التطبيق ا...
4              سيء جدا اسعار رسوم التوصيل لا تمت لواقع صله
                               ...                        
32031    التطبيق اصبح سيء لغايه نقوم بطلب لا يتم وصول ا...
32032                                           y love you
32033                 الباقه بتخلص وبشحن مرتين باقه اضافيه
32034    تطبيق فاشل وصلني الطلب ناقص ومش ينفع اعمل حاجه...
32035                                      ليش ما يفتح معي
Name: review_description, Length: 32036, dtype: object

# Install tensorflow_version

In [15]:
from re import X
try:
    %tensorflow_version 2.X
except Exception:
    pass
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#!pip install tensorflow_addons

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


# Tokenization and Embedding

In [5]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding

#regulatization
from tensorflow.keras.regularizers import l1_l2

X_train = train_data['review_description']
y_train = train_data['rating']
x_test = test_data['review_description'].astype(str).tolist()

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_tokenized = tokenizer.texts_to_sequences(X_train)
X_tokenized_test =tokenizer.texts_to_sequences(x_test)

# Padding sequences
max_len = max([len(x) for x in X_tokenized])
X_padded = pad_sequences(X_tokenized, maxlen=max_len, padding='post')
X_test_padded = pad_sequences(X_tokenized_test, maxlen=max_len, padding='post')
y_train = np.array(y_train)
# Encoding labels
y_train_encoded = y_train + 1  # Assuming 3 classes
y_train_encoded = to_categorical(y_train, num_classes=3)
#print(y_train_encoded)
#print(y_train_encoded.argmax(axis=-1))

# Predict the sentiments

In [ ]:
#prediction
def prediction_test(model):
  predictions = model.predict(X_test_padded)
  # Convert probabilities to class labels
  predicted_classes = predictions.argmax(axis=-1)
  predicted_classes = np.vectorize({2: -1, 1: 1, 0: 0}.get)(predicted_classes)  # To shift (0, 1, 2) to (-1, 0, 1)
  #predicted_labels = [np.argmax(pred) for pred in predicted_classes]
  print(predicted_classes)
  return predicted_classes

# Transformer Model

In [ ]:
#best accuracy
import tensorflow as tf
!pip install tensorflow_addons

import tensorflow_addons as tfa
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

import tensorflow_addons as tfa
import tensorflow_addons as tfa
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

import tensorflow_addons as tfa
class TransformerModel(tf.keras.Model):
    def _init_(self, vocab_size, d_model, num_heads, ff_dim, num_transformer_blocks, output_dim, rate=0.1):
        super(TransformerModel, self)._init_()
        self.embedding = layers.Embedding(vocab_size, d_model)
        self.attention = tfa.layers.MultiHeadAttention(head_size=num_heads, num_heads=num_heads)
        self.ffn = tf.keras.Sequential([
            layers.Dense(ff_dim, activation='relu'),
            layers.Dense(d_model),
        ])
        
        self.rnn_layer = layers.GRU(16, return_sequences=True)    # Added GRU  layer

        self.layer_norm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layer_norm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.pooling = layers.GlobalAveragePooling1D()
        self.dense = layers.Dense(output_dim, activation='softmax')

    def call(self, inputs):
      x = self.embedding(inputs)

      # Reshape x to (batch_size, sequence_length, num_features)
      x = tf.reshape(x, [-1, tf.shape(x)[1], self.embedding.output_dim])

      attention_output = self.attention([x, x, x])
      x = x + attention_output
      x = self.rnn_layer(x)
      #x = self.lstm_layer(x)  # Added LSTM layer
      x = self.layer_norm1(x)
      x = self.ffn(x)
      x = self.dropout1(x)
      x = x + attention_output
      x = self.layer_norm2(x)
      x = self.pooling(x)
      return self.dense(x)

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import layers
!pip install keras_tuner
import keras_tuner as kt

def build_transformer_model(hp):
    vocab_size = len(tokenizer.word_index) + 1
    # num_transformer_blocks = 2
    output_dim = 3  # Adjust based on your dataset

    # d_model = hp.Int('d_model', min_value=32, max_value=128, step=32)
    # num_heads = hp.Int('num_heads', min_value=128, max_value=512, step=128)
    # ff_dim = hp.Int('ff_dim', min_value=32, max_value=128, step=32)
    # dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)
    dropout_rate = hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)

    num_transformer_blocks = hp.Int('num_transformer_blocks', min_value=2, max_value=6, step=1)
    d_model = hp.Int('d_model', min_value=256, max_value=1024, step=256)
    num_heads = hp.Int('num_heads', min_value=4, max_value=12, step=2)
    ff_dim = hp.Int('ff_dim', min_value=512, max_value=2048, step=512)

    model = TransformerModel(vocab_size, d_model, num_heads, ff_dim, num_transformer_blocks, output_dim, rate=dropout_rate)
    # learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
    learning_rate = hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='log')

    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop']),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

tuner = kt.RandomSearch(
    build_transformer_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='transformer_tuning'
)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(
    X_padded, y_train_encoded,
    epochs=10,
    batch_size=16,
    validation_split=0.2,
    callbacks=[early_stopping]
)
# Retrieve the best model and hyperparameters
best_model1 = tuner.get_best_models(num_models=1)[0]
best_hyperparameters1 = tuner.get_best_hyperparameters()[0]

print('Best hyperparameters:', best_hyperparameters1.values)
predictions_ct = prediction_test(best_model1)

In [ ]:
import csv
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def CreateCompare_files(predicted,file_comparePath,file_create):
  # Read the CSV file into a DataFrame
  id = pd.read_csv("/content/test _no_label.csv", encoding="utf-8", dtype={'ID': int})
  print(len(id))
  # Assuming you have a column named 'ID' and 'Predicted_Label' in the DataFrame 'id'
  with open("test_with_label.csv", mode="w", newline="") as csvfile:
    fieldnames = ["ID", "rating"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for index, row in id.iterrows():
      writer.writerow({"ID": row["ID"], "rating": predicted[index]})
  y_eval=pd.read_csv(file_comparePath, encoding="utf-8", dtype={'rating': int})
  accuracy = accuracy_score(y_eval["rating"],predicted_classes)
  precision = precision_score(y_eval["rating"],predicted_classes, average='weighted')
  recall = recall_score(y_eval["rating"],predicted_classes, average='weighted')
  f1 = f1_score(y_eval["rating"],predicted_classes, average='weighted')
  # Print the evaluation metrics
  print("Accuracy:", accuracy)
  print("Precision:", precision)
  print("Recall:", recall)
  print("F1-score:", f1)

#file best accuray "/content/test_with_label (2).csv" to compare
#file accuray "test_with_label.csv" to create

CreateCompare_files(predictions_ct,"/content/test_with_label (2).csv","test_with_label.csv")


# LSTM Model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
!pip install keras_tuner
import keras_tuner as kt
# Prepare the LSTM model
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 500

# Convert labels to categorical

# Define your LSTM model
def build_lstm_model(hp):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=500, input_length=max_len))
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=128, step=32), return_sequences=True))
    model.add(Dropout(rate=hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(Bidirectional(LSTM(units=hp.Int('units_bidi', min_value=32, max_value=128, step=32))))
    model.add(Dropout(rate=hp.Float('dropout_bidi', min_value=0.1, max_value=0.5, step=0.1)))

    # New Dense layers with hyperparameter tuning
    model.add(Dense(units=hp.Int('dense_units_1', min_value=8, max_value=64, step=8), activation=hp.Choice('dense_activation_1', values=['relu', 'sigmoid'])))
    model.add(Dropout(rate=hp.Float('dense_dropout_1', min_value=0.1, max_value=0.5, step=0.1)))

    model.add(Dense(units=hp.Int('dense_units_2', min_value=4, max_value=32, step=4), activation=hp.Choice('dense_activation_2', values=['relu', 'sigmoid'])))
    model.add(Dropout(rate=hp.Float('dense_dropout_2', min_value=0.1, max_value=0.5, step=0.1)))

    model.add(Dense(3, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

tuner = kt.RandomSearch(
    build_lstm_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='lstm_tuning'
)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(
    X_padded, y_train_encoded,
    epochs=10,
    batch_size=16,
    validation_split=0.2,
    callbacks=[early_stopping]
)


# Retrieve the best model from tuner
best_lstm_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters()[0]

print('Best hyperparameters:', best_hyperparameters.values)